# LONG SHORT TERM MEMORY

### We use this to solve the problem of vanishing gradients

In [1]:
# importing libraries

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import MNIST

In [2]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.1307,))
])

In [3]:
#MNIST Dataset

train_dataset = MNIST(
    root='./data',
    train=True,
    transform=transforms,
    download=True
)

test_dataset = MNIST(
    root='./data',
    train=False,
    transform=transforms
)

Failed to download (trying next):
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>



100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>



100.0%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [4]:
num_classes = 10
batch_size = 2
num_epochs = 2
learning_rate = 0.01

### train and test loader

In [6]:
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

### Defining the Model

In [7]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, 
                            hidden_size,
                            num_layers,
                            batch_first=True)
        self.fcl = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        
        # output from prev block
        h0 = torch.zeros(self.num_layers,
                         x.size(0),
                         self.hidden_size)

        # memory from prev block
        c0 = torch.zeros(self.num_layers,
                         x.size(0),
                         self.hidden_size)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fcl(out[:, -1, :])
        return out

In [8]:
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2

### Training the Model

In [10]:
model = RNN(input_size, hidden_size, num_layers, num_classes)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
            print(f"Epoch {epoch+1}, step {i+1}/{total_steps}, loss = {loss.item()}")
        
        

Epoch 1, step 100/30000, loss = 2.240078926086426
Epoch 1, step 200/30000, loss = 1.3520734310150146
Epoch 1, step 300/30000, loss = 1.8166661262512207
Epoch 1, step 400/30000, loss = 0.23635633289813995
Epoch 1, step 500/30000, loss = 2.2122409343719482
Epoch 1, step 600/30000, loss = 0.7011798024177551
Epoch 1, step 700/30000, loss = 0.35173436999320984
Epoch 1, step 800/30000, loss = 2.6870386600494385
Epoch 1, step 900/30000, loss = 1.3084716796875
Epoch 1, step 1000/30000, loss = 1.177424430847168
Epoch 1, step 1100/30000, loss = 1.1538671255111694
Epoch 1, step 1200/30000, loss = 0.34378698468208313
Epoch 1, step 1300/30000, loss = 0.21705982089042664
Epoch 1, step 1400/30000, loss = 0.24881631135940552
Epoch 1, step 1500/30000, loss = 1.2029935121536255
Epoch 1, step 1600/30000, loss = 1.4119383096694946
Epoch 1, step 1700/30000, loss = 0.35807013511657715
Epoch 1, step 1800/30000, loss = 1.425188660621643
Epoch 1, step 1900/30000, loss = 1.9035402536392212
Epoch 1, step 2000/30

### Testing the Model

In [11]:
correct = total = 0

for images, labels in test_loader:
    images = images.reshape(-1, sequence_length, input_size)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
print(f"test accuracy = {100*correct/total}")

test accuracy = 85.87
